### Keras implementation of Brain CNN

In [1]:
import tensorflow as tf
import numpy as np
import sklearn.metrics


from keras.utils.np_utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D, Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
IMG_WIDTH = 64
IMG_HEIGHT = 64
SNAPS = 8

SLICE = 5
CLASSES = 2
CHANNELS = 1

conv1_filter = 4
conv2_filter = 8
conv3_filter = 16
conv4_filter = 32
conv5_filter = 4
conv6_filter = 4

In [3]:
experiment = '1snap3d'
path = '/work/aaung/datasets/' + experiment + '/'

_04847_img = np.load(path + '4847_' + experiment + '-image.npy')
_04799_img = np.load(path + '4799_' + experiment + '-image.npy')
_04820_img = np.load(path + '4820_' + experiment + '-image.npy')
_05675_img = np.load(path + '5675_' + experiment + '-image.npy')
_05680_img = np.load(path + '5680_' + experiment + '-image.npy')
_05710_img = np.load(path + '5710_' + experiment + '-image.npy')

_04847_lbl = np.load(path + '4847_' + experiment + '-label-onehot.npy')
_04799_lbl = np.load(path + '4799_' + experiment + '-label-onehot.npy')
_04820_lbl = np.load(path + '4820_' + experiment + '-label-onehot.npy')
_05675_lbl = np.load(path + '5675_' + experiment + '-label-onehot.npy')
_05680_lbl = np.load(path + '5680_' + experiment + '-label-onehot.npy')
_05710_lbl = np.load(path + '5710_' + experiment + '-label-onehot.npy')

### Leave on example out validation

In [4]:
# n = 24
train_img = np.vstack((_04847_img[n:,], _04799_img[n:,], _04820_img[n:,], _05675_img[n:,], _05680_img[n:,], _05710_img[n:,]))
train_lbl = np.vstack((_04847_lbl[n:,], _04799_lbl[n:,], _04820_lbl[n:,], _05675_lbl[n:,], _05680_lbl[n:,], _05710_lbl[n:,]))
val_img = np.vstack((_04847_img[:n,], _04799_img[:n,], _04820_img[:n,], _05675_img[:n,], _05680_img[:n,], _05710_img[:n,]))
val_lbl = np.vstack((_04847_lbl[:n,], _04799_lbl[:n,], _04820_lbl[:n,], _05675_lbl[:n,], _05680_lbl[:n,], _05710_lbl[:n,]))

# Cross Subject
# train_img = np.vstack((_05710_img, _04847_img, _04799_img, _05675_img, _05680_img))
# train_lbl = np.vstack((_05710_lbl, _04847_lbl, _04799_lbl, _05675_lbl, _05680_lbl))
# val_img =  _04820_img
# val_lbl =  _04820_lbl 

NameError: name 'n' is not defined

In [ ]:
STRIP_HEIGHT = train_img.shape[2]
STRIP_WIDTH = train_img.shape[3]

In [ ]:
print train_img.shape
print val_img.shape
print train_lbl.shape
print val_lbl.shape

In [ ]:
np.random.seed(0)
# shuffle = np.random.permutation(database.shape[0])

# test = database[shuffle[0:100],:]
# val = database[shuffle[100:200],:]
# train = database[shuffle[200:],:]

xtrain = train_img[:,SLICE,:,:]
xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[1], xtrain.shape[2], 1))
ytrain = train_lbl

xval = val_img[:,SLICE,:,:]
xval = np.reshape(xval, (xval.shape[0], xval.shape[1], xval.shape[2], 1))
yval = val_lbl

In [ ]:
print xtrain.shape
print ytrain.shape
print xval.shape
print yval.shape

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

for i in range(5):
    plt.show(plt.imshow(xtrain[i,:,:,0]))

In [ ]:
class EvaluateValidation(Callback):      
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nValidation loss: {}, acc: {}\n'.format(loss, acc))

In [ ]:
### Model ###                                                                                                                               
model = Sequential()
mde = 0                                                                                                                                                                      
k_init = 'he_normal'
ridge = 0.0005

model.add(Convolution2D(conv1_filter, kernel_size=(3, 3), strides=(1, 1),
                        padding='same', data_format="channels_last", activation=None, use_bias=True,
                        kernel_regularizer=l2(ridge),
                        kernel_initializer=k_init, bias_initializer='zeros', input_shape=(STRIP_HEIGHT, STRIP_WIDTH, CHANNELS)))                                                                                                   
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(conv2_filter, kernel_size=(5, 5), strides=(1, 1),
                        padding='same', data_format="channels_last", activation=None, use_bias=True,
                        kernel_regularizer=l2(ridge),
                        kernel_initializer=k_init, bias_initializer='zeros'))                                                                                                   
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2), padding='valid'))

model.add(Convolution2D(conv3_filter, kernel_size=(7, 7), strides=(1, 1),
                        padding='same', data_format="channels_last", activation=None, use_bias=True,
                        kernel_regularizer=l2(ridge),
                        kernel_initializer=k_init, bias_initializer='zeros'))                                                                                                   
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Convolution2D(conv4_filter, kernel_size=(9, 9), strides=(1, 1),
                        padding='same', data_format="channels_last", activation=None, use_bias=True,
                        kernel_regularizer=l2(ridge),
                        kernel_initializer=k_init, bias_initializer='zeros'))                                                                                                   
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size = (2, 2), strides=2, padding='valid'))

model.add(Flatten())
model.add(Dense(1024, kernel_initializer=k_init,kernel_regularizer=l2(ridge)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(512, kernel_initializer=k_init,kernel_regularizer=l2(ridge)))
# model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(256, kernel_initializer=k_init,kernel_regularizer=l2(ridge)))
# model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(CLASSES, kernel_initializer=k_init,kernel_regularizer=l2(ridge)))
# model.add(BatchNormalization())
model.add(Activation('softmax'))

Lr = 1e-4 
dcy = 1e-5                                                                                                                                                                                
m = 0.5
batch_sz = 25
epoch = 25
# sgd = SGD(lr=Lr, momentum=m, decay=dcy,  nesterov=True)
adam = Adam(lr=Lr, decay=dcy)

model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.summary()
print('learning rate: %f, decay: %f' %(Lr, dcy))


from keras.backend import get_session
get_session().run(tf.global_variables_initializer())
a = model.fit(xtrain, ytrain, batch_size = batch_sz, epochs= epoch, verbose = 1,
              callbacks=[EvaluateValidation((xval, yval))])

loss_and_metrics = model.evaluate(xval, yval, batch_size=batch_sz)

print "Loss and accuracy: ", loss_and_metrics

In [ ]:
y_pred_one_hot = model.predict(xval, batch_size=128)
y_pred = np.argmax(y_pred_one_hot, axis=1)

In [ ]:
y_true = np.argmax(yval, axis=1)

In [ ]:
print "Test loss: {}".format(loss_and_metrics[0])
print "Test Acc: {} %".format(loss_and_metrics[1] * 100)
print "Precision", sklearn.metrics.precision_score(y_true, y_pred)
print "Recall", sklearn.metrics.recall_score(y_true, y_pred)
print "f1_score", sklearn.metrics.f1_score(y_true, y_pred)
print "confusion_matrix"
print sklearn.metrics.confusion_matrix(y_true, y_pred, labels=[0, 1])
fpr, tpr, tresholds = sklearn.metrics.roc_curve(y_true, y_pred)
ras = sklearn.metrics.auc(fpr, tpr)
roauc_score = sklearn.metrics.roc_auc_score(y_true, y_pred)
print ras
print roauc_score

In [ ]:
print "{}".format(loss_and_metrics[0])
print "{}".format(loss_and_metrics[1] * 100)
print sklearn.metrics.precision_score(y_true, y_pred)
print sklearn.metrics.recall_score(y_true, y_pred)
print sklearn.metrics.f1_score(y_true, y_pred)
print sklearn.metrics.confusion_matrix(y_true, y_pred, labels=[0, 1])



# Compute ROC curve and ROC area for each class
n_classes = 2
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = sklearn.metrics.roc_curve(yval[:, i], y_pred_one_hot[:, i])
    roc_auc[i] = sklearn.metrics.auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = sklearn.metrics.roc_curve(yval.ravel(), y_pred_one_hot.ravel())
roc_auc["micro"] = sklearn.metrics.auc(fpr["micro"], tpr["micro"])

roauc_score = sklearn.metrics.roc_auc_score(y_true, y_pred)

print roauc_score


In [ ]:
plt.figure()
lw = 2
plt.plot(fpr["micro"], tpr["micro"], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc["micro"])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# plt.title('ROC classifying Faces of class {} (Pure class {} vs Mixed class {})'.format(CLASS, CLASS, CLASS))
plt.legend(loc="lower right")
plt.show()

In [ ]:
print tpr["micro"]
print fpr["micro"]